# Este código limpia los datos de contaminación lumínica por AGEB en 2019 y 2020

Los datos se obtuvieron a través de la página https://eogdata.mines.edu/products/vnl/. Despúes fueron tratados en QGIS para asociarlos a cada unidad de observación. 

In [16]:
import os
import pandas as pd

In [17]:
# Carpeta de origen con los archivos crudos de luz nocturna
carpeta = r"C:\Users\santo\Documents\CIDE\Tesis\datos_luz"

# Diccionario año : nombre de archivo
files = {
    '2019': 'luz_2019.csv',
    '2020': 'luz_2020.csv'
}

# Lista de meses que se duplicarán.
#  '03'  → marzo,  '05' → mayo, etc.
months_to_keep = ['03']

In [18]:
# ------------------------------------------------------------------
# 2.  FUNCIÓN AUXILIAR PARA CREAR UNA COPIA POR MES
# ------------------------------------------------------------------
def build_for_month(month_code: str) -> pd.DataFrame:
    """
    Genera un DataFrame concatenado (2019 y 2020) con el mes especificado
    en formato YYYY-MM.  Devuelve columnas: CVEGEO, light_mean, mes
    """
    frames = []
    for year, fname in files.items():
        path = os.path.join(carpeta, fname)
        df = (
            pd.read_csv(path, usecols=['CVEGEO', '_light_mean'])
              .rename(columns={'_light_mean': 'light_mean', 'CVEGEO': 'poligono_final'})
              .assign(mes=f"{year}-{month_code}")
        )
        frames.append(df)
    return pd.concat(frames, ignore_index=True)

# ------------------------------------------------------------------
# 3.  CONSTRUIR EL DATAFRAME ÚNICO
# ------------------------------------------------------------------
dfs = [build_for_month(m) for m in months_to_keep]
luz_limpio_full = pd.concat(dfs, ignore_index=True)

In [19]:
# ------------------------------------------------------------------
# 4.  EXPORTAR A CSV
# ------------------------------------------------------------------

# Carpeta de destino donde queremos dejar la versión limpia
carpeta_final = r"C:\Users\santo\Documents\CIDE\Tesis\datos_unidos"
luz_limpio_full.to_csv(os.path.join(carpeta_final, 'luz_limpio.csv'), index=False, encoding='utf-8-sig')

## Datos de luz mensuales

Esta sección limpiará los datos de luz mensuales

In [20]:
import os
import re #sirve para las expresiones regulares para validar y extraer año/mes del nombre del archivo.
import pandas as pd

In [21]:
#Carpeta de origen y de salida de los archivos
DIR_IN  = r"C:\Users\santo\Documents\CIDE\Tesis\datos_luz\monthly\csv"
DIR_OUT = r"C:\Users\santo\Documents\CIDE\Tesis\datos_unidos"

# patrón esperado: luz_YYYY_MM.csv
pat = re.compile(r"^luz_(\d{4})_(\d{2})\.csv$", flags=re.IGNORECASE)
#En esa linea estamos compilando un patrón regex para nombres de archivo luz_YYYY_MM.csv 
#^:inicio de la cadena 
#(\d{4}): grupo de 4 dígitos
#\.csv : que literalmente eté escrito .csv (el punto se escapa con \.)
#re.IGNORECASE : no distingue mayúsculas/minúsculas

# columnas que queremos conservar del CSV mensual
KEEP_COLS = ["CVEGEO", "light_monthly_mean", "light_monthly_sum"]

frames = []
files = sorted(os.listdir(DIR_IN)) #obtiene los nombres de todos los archivos de la carpeta y los ordena alfabéticamente.

In [22]:
for fname in files:
    m = pat.match(fname) #intenta encajar el nombre contra el patrón regex
    if not m:
        # salta cualquier archivo que no cumpla el patrón
        continue

    year, month = m.group(1), m.group(2) #extrae año y mes de los grupos capturados por la regex
    fpath = os.path.join(DIR_IN, fname) #arma la ruta completa al archivo

    # leer como texto para no perder ceros a la izquierda
    df = pd.read_csv(fpath, dtype="object")

    # validar columnas
    missing = [c for c in KEEP_COLS if c not in df.columns] #detecta si faltan columnas en el csv
    if missing:
        raise ValueError(f"{fname} no tiene columnas requeridas: {missing}")

    # Renombrar columna
    df = df[KEEP_COLS].rename(columns={"CVEGEO": "poligono_final"})
    df["poligono_final"] = df["poligono_final"].astype(str) #asegura que poligono_final sea texto y sin espacios alrededor.

    # convertir columnas a numéricas y los valores no convertibles e vuelven Nan
    df["light_monthly_mean"] = pd.to_numeric(df["light_monthly_mean"], errors="coerce")
    df["light_monthly_sum"]  = pd.to_numeric(df["light_monthly_sum"], errors="coerce") 

    # agregar columna mes
    df["mes"] = f"{year}-{month}"

    # limpieza básica (elimina filas sin poligono_final y elimina duplicados para la clave (poligono_final, mes)
    df = df.dropna(subset=["poligono_final"]).drop_duplicates(
        subset=["poligono_final", "mes"]
    )

    frames.append(df) #agrega el dataframe del mes actual a la lista frames para después unir todo 

if not frames:
    raise RuntimeError("No se encontraron archivos luz_YYYY_MM.csv en la carpeta indicada.")

panel = pd.concat(frames, ignore_index=True).sort_values(["mes","poligono_final"])

# (opcional) chequeo de consistencia por mes
check = panel.groupby("mes").agg(
    n_ageb=("poligono_final","nunique")
).reset_index()
print(check.head())


       mes  n_ageb
0  2019-03    2454
1  2019-04    2454
2  2019-05    2454
3  2019-06    2454
4  2019-07    2454


In [23]:
# guardar el archivo
OUTFILE = os.path.join(DIR_OUT, "luz_monthly_limpio.csv")
panel.to_csv(OUTFILE, index=False, encoding="utf-8-sig")
print(f"✅ Panel guardado en: {OUTFILE}")
print(f"Filas: {len(panel)}, Meses: {panel['mes'].nunique()}, AGEBs únicos: {panel['poligono_final'].nunique()}")


✅ Panel guardado en: C:\Users\santo\Documents\CIDE\Tesis\datos_unidos\luz_monthly_limpio.csv
Filas: 31902, Meses: 13, AGEBs únicos: 2454
